# Quick intro to Shapely

This jupyter notebook is heavily based on Sebastian Hohmann's GIS course

Contents:
- [1 Overview](#1-Overview)
- [2 Points](#2-Points)
  - [2.1 Defining points](#2.1-Defining-points)
  - [2.2 Getting co-ordinates](#2.2-Getting-co-ordinates)
  - [2.3 Calculating distances between points](#2.3-Calculating-distances-between-points)
- [3 Lines](#3-Lines)
  - [3.1 Defining lines](#3.1-Defining-lines)
  - [3.2 Getting co-ordinates, computing length](#3.2-Getting-co-ordinates,-computing-length)
- [4 Polygons](#4-Polygons)
  - [4.1 Defining Polygons](#4.1-Defining-Polygons)
  - [4.2 Getting bounds, envelope, exterior, area, centroid](#4.2-Getting-bounds,-envelope,-exterior,-area,-centroid)
    - [4.2.1 bounds](#4.2.1-bounds)
  - [4.2.2 envelope](#4.2.2-envelope)
    - [4.2.3 exterior](#4.2.3-exterior)
    - [4.2.4 area](#4.2.4-area)
    - [4.2.5 centroid](#4.2.5-centroid)
- [5 Geometry collections](#5-Geometry-collections)

In [ ]:
from shapely.geometry import Point, LineString, Polygon

# 1 Overview

[Shapely](https://shapely.readthedocs.io/en/stable/manual.html) is a library for geometric objects. Fundamental types are
- `Points`
- `Lines`
- `Polygons`

Shapely can 
- create `Lines` or `Polygons` from `Collections` of `Points`
- calculate things like lengths and areas
- do geometric operations like `Union`, `Difference`, `Distance`
- do spatial queries between geometries like `Intersects`, `Touches`, `Within`

Geometric objects are tuples
- `Points` can be 2-D (x, y) or 3-D (x, y, z)
- `LineString` are sequence of points joined together (i.e. at least two co-ordinate tuples)
- `Polygon` filled area of at least 3 co-ordinate tuples

Geometric collections
- `MultiPoint`, `MultiLineString` and `MultiPolygon` are collections of `Point`, `LineString` and `Polygon` objects (i.e. they consist of multiple parts of the these types of objects)

# 2 Points

## 2.1 Defining points

In [ ]:
p1 = Point(1, 1)
p2 = Point(2, 3)
p3 = Point(4, -2)

In [ ]:
p1

In [ ]:
type(p1)

In [ ]:
p1.geom_type

## 2.2 Getting co-ordinates

In [ ]:
p2.coords.xy

In [ ]:
p2.x

In [ ]:
p2.y

## 2.3 Calculating distances between points

In [ ]:
p1.distance(p2)

we can also do this manually

In [ ]:
((p1.x - p2.x) ** 2 + (p1.y - p2.y) ** 2) ** 0.5

# 3 Lines

## 3.1 Defining lines

Lines can be defined from 
- coordinates
- `Point` objects

In [ ]:
l1 = LineString([p1, p2, p3])
l2 = LineString([(3.2, 1.6), (-2.7, 8.2)])

In [ ]:
l1

In [ ]:
l2

In [ ]:
l1.geom_type

## 3.2 Getting co-ordinates, computing length

In [ ]:
l1.xy

In [ ]:
l1.xy[0]

In [ ]:
l1.length

Once again, we can use the pythagorean theorem to compute the length ourselves

In [ ]:
(((l1.xy[0][0] - l1.xy[0][1]) ** 2 +
 (l1.xy[1][0] - l1.xy[1][1]) ** 2) ** 0.5 +
 ((l1.xy[0][1] - l1.xy[0][2]) ** 2 +
 (l1.xy[1][1] - l1.xy[1][2]) ** 2) ** 0.5)

# 4 Polygons

## 4.1 Defining Polygons

Once again, we can define polygons from existing `Point` objects or by listing their vertices

In [ ]:
pol1 = Polygon([p1, p2, p3])
pol2 = Polygon([(3.2, 1.6), (-2.7, 8.2), (5.5, 5.5), (8.2, 8.2)])

In [ ]:
pol1

In [ ]:
pol2

In [ ]:
print(pol1)

Note the double parentheses. this is because polygons can have holes:

In [ ]:
exterior = [(-180, 90), (-180, -90), (180, -90), (180, 90)]
hole = [(x*0.8, y*0.8) for x, y in exterior]
exterior_poly = Polygon(exterior)
poly_with_hole = Polygon(shell=exterior, holes=[hole])

In [ ]:
poly_with_hole

In [ ]:
print(poly_with_hole)

## 4.2 Getting bounds, envelope, exterior, area, centroid

### 4.2.1 bounds

Recall that we had defined `pol1` from `p1`, `p2`, `p3`. 

In [ ]:
for p in [p1, p2, p3]:
    print(p.xy)

In [ ]:
pol1.bounds

In [ ]:
?pol1.bounds

## 4.2.2 envelope

In [ ]:
pol1.envelope

In [ ]:
?pol1.envelope

### 4.2.3 exterior

In [ ]:
pol1.exterior

In [ ]:
pol1.exterior.xy

In [ ]:
pol1.envelope.exterior.xy

### 4.2.4 area

In [ ]:
pol1.area

This is a simple triangle, so we could easily find its area. For more general shapes, there is the [Shoelace Formula](https://en.wikipedia.org/wiki/Shoelace_formula)

In [ ]:
def area_shoelace(p, is_area=True):
    x, y = p.exterior.xy[0], p.exterior.xy[1]
    A = 0.5 * sum([x[i]*y[i+1] - x[i+1]*y[i] for i in range(len(x)-1)])
    if is_area:
        return abs(A)
    else:
        return A

In [ ]:
area_shoelace(pol1)

In [ ]:
pol2.area

In [ ]:
area_shoelace(pol2)

### 4.2.5 centroid

In [ ]:
pol1.centroid

In [ ]:
pol1.centroid.xy

In [ ]:
pol2.centroid.xy

Once again, we can calculate this from scratch

In [ ]:
def centroid_from_scratch(p):
    
    '''
    https://en.wikipedia.org/wiki/Centroid#Of_a_polygon
    '''
    x, y = p.exterior.xy[0], p.exterior.xy[1]
    A = area_shoelace(p, is_area=False)
    Cx = (1/(6*A)) * sum([(x[i] + x[i+1])*(x[i]*y[i+1] - x[i+1]*y[i]) for i in range(len(x)-1)])
    Cy = (1/(6*A)) * sum([(y[i] + y[i+1])*(x[i]*y[i+1] - x[i+1]*y[i]) for i in range(len(x)-1)])
    return Cx, Cy

In [ ]:
centroid_from_scratch(pol1)

In [ ]:
centroid_from_scratch(pol2)

# 5 Geometry collections

We can have collections of `Point`s, `LineString`s and `Polygon`s in a GeoPandas `geometry` field. So it is good to know about them

In [ ]:
from shapely.geometry import MultiPoint, MultiLineString, MultiPolygon

In [ ]:
mpt = MultiPoint([p1, p2, p3])
mln = MultiLineString([l1, l2])
mpo = MultiPolygon([pol1, pol2])

In [ ]:
mpt

In [ ]:
print(mpt)

In [ ]:
mln

In [ ]:
print(mln)

In [ ]:
mpo

In [ ]:
print(mpo)

One nice method to know is the `convex_hull`. We can call this on any geometry collection

In [ ]:
mpt.convex_hull

In [ ]:
mln.convex_hull

In [ ]:
mpo.convex_hull

One important property for `MultiPolygon`s is whether they are **valid** -- if their components do not intersect one another.

In [ ]:
mpo.is_valid

Let's quickly define one that **is** valid

In [ ]:
pol3 = Polygon([(3.2, 2.8), (-2.7, 8.2), (5.5, 5.5), (8.2, 8.2)])
mpo2 = MultiPolygon([pol1, pol3])
mpo2

In [ ]:
mpo2.is_valid